In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.transforms import functional as F
from PIL import Image
from torch import nn

# Define the ESRGAN Model (RRDBNet)
class RRDBNet(nn.Module):
    def __init__(self, in_channels, out_channels, nf, nb, gc=32):
        super(RRDBNet, self).__init__()
        RRDB_block_f = lambda nf, gc: RRDB(nf, gc)
        self.conv_first = nn.Conv2d(in_channels, nf, 3, 1, 1, bias=True)
        self.RRDB_trunk = self.make_layer(RRDB_block_f, nb, nf, gc)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_channels, 3, 1, 1, bias=True)

    def make_layer(self, block, num_blocks, *args):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(*args))
        return nn.Sequential(*layers)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea += trunk
        fea = torch.nn.functional.interpolate(fea, scale_factor=2, mode="nearest")
        fea = torch.nn.functional.interpolate(fea, scale_factor=2, mode="nearest")
        fea = self.conv_last(fea)
        return fea


# Load the ESRGAN Model
def load_esrgan_model(model_path="RRDB_ESRGAN_x4.pth"):
    model = RRDBNet(3, 3, 64, 23, gc=32)
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model


# Upscale the image with ESRGAN
def upscale_image_with_esrgan(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = F.to_tensor(Image.fromarray(image)).unsqueeze(0)
    with torch.no_grad():
        output_tensor = model(input_tensor).squeeze(0)
    output_image = F.to_pil_image(output_tensor.clamp(0, 1))
    return np.array(output_image)


# Paths to the images
input_image_path = "./generate_images/bad_m2_1.png"
master_image_path = "./Meter.jpg"

# Load the images
input_image = cv2.imread(input_image_path)
master_image = cv2.imread(master_image_path)

# Load the ESRGAN model
esrgan_model = load_esrgan_model("RRDB_ESRGAN_x4.pth")  # Path to the ESRGAN model

# Upscale the images
input_image_hr = upscale_image_with_esrgan(input_image, esrgan_model)
master_image_hr = upscale_image_with_esrgan(master_image, esrgan_model)

# Continue with grayscale conversion, thresholding, and contour detection
input_gray = cv2.cvtColor(input_image_hr, cv2.COLOR_BGR2GRAY)
master_gray = cv2.cvtColor(master_image_hr, cv2.COLOR_BGR2GRAY)

# Thresholding
_, input_thresh = cv2.threshold(input_gray, 128, 255, cv2.THRESH_BINARY_INV)
_, master_thresh = cv2.threshold(master_gray, 128, 255, cv2.THRESH_BINARY_INV)

# Contour Detection
input_contours, _ = cv2.findContours(input_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
master_contours, _ = cv2.findContours(master_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Bounding Box Extraction
def get_segment_bounding_boxes(contours):
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))
    return bounding_boxes


input_bounding_boxes = get_segment_bounding_boxes(input_contours)
master_bounding_boxes = get_segment_bounding_boxes(master_contours)

# Matching Segments with Tolerance
position_tolerance = 20
size_tolerance = 0.14

def is_segment_present(master_box, input_boxes, position_tolerance, size_tolerance):
    mx, my, mw, mh = master_box
    for ix, iy, iw, ih in input_boxes:
        if (abs(mx - ix) <= position_tolerance and
            abs(my - iy) <= position_tolerance and
            abs(mw - iw) / mw <= size_tolerance and
            abs(mh - ih) / mh <= size_tolerance):
            return True
    return False


# Check and Mark Missing Segments
all_segments_present = True
for master_box, master_contour in zip(master_bounding_boxes, master_contours):
    if not is_segment_present(master_box, input_bounding_boxes, position_tolerance, size_tolerance):
        all_segments_present = False
        cv2.drawContours(master_image_hr, [master_contour], -1, (0, 0, 255), 2)

if all_segments_present:
    print("All segments in the master image are present in the input image.")
else:
    print("Some segments in the master image are missing in the input image.")

# Display Results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Input Image (High Resolution)")
plt.imshow(cv2.cvtColor(input_image_hr, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Master Image with Missing Segments Marked")
plt.imshow(cv2.cvtColor(master_image_hr, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()


(904, 340)
(1652, 604)


True

In [133]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cache the ESRGAN model
esrgan_model = None

def load_esrgan_model():
    global esrgan_model
    if esrgan_model is None:
        esrgan_model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
    return esrgan_model

In [134]:
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    return image

def apply_super_resolution(image):
    # Load the ESRGAN model
    model = load_esrgan_model()
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Convert the image to a tensor
    image_tensor = tf.convert_to_tensor(image_rgb, dtype=tf.float32)
    image_tensor = tf.expand_dims(image_tensor, 0)
    # Apply the ESRGAN model
    sr_image_tensor = model(image_tensor)
    sr_image = tf.squeeze(sr_image_tensor).numpy()
    # Convert the image back to grayscale
    sr_image_gray = cv2.cvtColor(sr_image.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    return sr_image_gray

In [135]:

def find_contours(image, threshold_value):
    # Increase image contrast
    alpha = 1.5  # Contrast control (1.0-3.0)
    beta = 0     # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    # Apply Gaussian blur to reduce noise and improve contour detection
    blurred = cv2.GaussianBlur(adjusted, (5, 5), 0)

    # Apply morphological operations
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)

    # Use the morphed image for thresholding and contour detection
    _, binary = cv2.threshold(morph, threshold_value, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_detected = []
    for contour in contours:
        if cv2.contourArea(contour) > 10:
            contours_detected.append(contour)
    return contours_detected

def draw_contours(image, contours):
    # Draw contours on the image
    image_with_contours = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)
    return image_with_contours

def draw_missing_segments(master_contours, captured_contours, master_image):
    # Compare contours to find missing segments
    missing_contours = []
    for master_contour in master_contours:
        found = False
        for captured_contour in captured_contours:
            if cv2.matchShapes(master_contour, captured_contour, cv2.CONTOURS_MATCH_I1, 0.0) < 0.1:  # Shape matching threshold
                found = True
                break
        if not found:
            missing_contours.append(master_contour)

    # Draw missing segments
    for contour in missing_contours:
        cv2.drawContours(master_image, [contour], -1, (0, 0, 255), 2)

In [136]:
master_image_path = '../Analog/train/class 1/good_1.png'
captured_image_path = '../Analog/train/class 0/bad_m_576.png'

# Preprocess the images
try:
    master_image = preprocess_image(master_image_path)
    captured_image = cv2.imread(captured_image_path)
    if captured_image is None:
        raise ValueError(f"Image at path {captured_image_path} could not be loaded.")
    captured_image_gray = preprocess_image(captured_image_path)
except ValueError as e:
    print(e)
    exit(1)

# Apply super-resolution
master_image_sr = apply_super_resolution(master_image)
captured_image_sr = apply_super_resolution(captured_image_gray)

print(master_image_sr.shape)
print(captured_image_sr.shape)

# Convert the captured image to BGR for threshold selection
captured_image_bgr = cv2.cvtColor(captured_image_gray, cv2.COLOR_GRAY2BGR)

# Select the threshold value dynamically
threshold_value = select_threshold(captured_image_bgr)

# Find contours
master_contours = find_contours(master_image_sr, threshold_value)
captured_contours = find_contours(captured_image_sr, threshold_value)

# Draw contours on the images
master_image_with_contours = draw_contours(master_image_sr, master_contours)
captured_image_with_contours = draw_contours(captured_image_sr, captured_contours)

# Draw missing segments
draw_missing_segments(master_contours, captured_contours, master_image_sr)

# Resize images for better visibility
master_image_resized = cv2.resize(master_image_with_contours, (150, 418))
captured_image_resized = cv2.resize(captured_image_with_contours, (150, 418))
captured_image_with_missing_segments_resized = cv2.resize(master_image_sr, (150, 418))

# Display the images
cv2.imshow('Master Image with Contours', master_image_resized)
cv2.imshow('Captured Image with Contours', captured_image_resized)
cv2.imshow('Captured Image with Missing Segments', captured_image_with_missing_segments_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the results
cv2.imwrite('master_image_with_contours.jpg', master_image_with_contours)
cv2.imwrite('captured_image_with_contours.jpg', captured_image_with_contours)
cv2.imwrite('captured_image_with_missing_segments.jpg', captured_image)

(904, 340)
(1652, 604)


True